#Sentiment Analysis With VADER model on twitter reviews

In [1]:
import pandas as pd
import nltk

In [2]:
#Download vader lexicon 
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
df=pd.read_csv("/content/1592918410_Update1.csv")

In [4]:
df.head()

,Unnamed: 0,airline_sentiment,name,text,tweet_created,tweet_location,user_timezone,hashtags,tags
0,0,neutral,cairdin,virginamerica dhepburn said,24/02/15 11:35,NaN,Eastern Time (US & Canada),[],"['virginamerica', 'dhepburn']"
1,1,positive,jnardino,virginamerica plus youve added commercials exp...,24/02/15 11:15,NaN,Pacific Time (US & Canada),[],['virginamerica']
2,2,neutral,yvonnalynn,virginamerica didnt today must mean need take ...,24/02/15 11:15,Lets Play,Central Time (US & Canada),[],['virginamerica']
3,3,negative,jnardino,virginamerica really aggressive blast obnoxiou...,24/02/15 11:15,NaN,Pacific Time (US & Canada),[],['virginamerica']
4,4,negative,jnardino,virginamerica really big bad thing,24/02/15 11:14,NaN,Pacific Time (US & Canada),[],['virginamerica']


In [5]:
df.shape

(14640, 9)

In [6]:
df.columns

Index(['Unnamed: 0', 'airline_sentiment', 'name', 'text', 'tweet_created',
       'tweet_location', 'user_timezone', 'hashtags', 'tags'],
      dtype='object')

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
# we make instance of SentimentIntensityAnalyzer
sid=SentimentIntensityAnalyzer()

In [9]:
# Example :- 1
a="This is a good movie"
sid.polarity_scores(a)

{'compound': 0.4404, 'neg': 0.0, 'neu': 0.508, 'pos': 0.492}



1.   neg= Negative Values
2.   neu= Neutral Values
3.   pos= Positive Values
4.   compund=normalazation of Negative value,neutral Value and Positive Value
**Note:- the max value of all of these is 1.0**



In [10]:
# Example:-2
b="This was the best ,most awesome movie EVER MADE!!!!"
sid.polarity_scores(b)

{'compound': 0.8948, 'neg': 0.0, 'neu': 0.418, 'pos': 0.582}

**Note:-** Vader is quite enough to understand thinks like repeated Puctuation and capitalization.

In [11]:
#Example 3 which is so quite negative review
c="This was the WORST Movie that has ever disgraced the screen "
sid.polarity_scores(c)

{'compound': -0.8331, 'neg': 0.465, 'neu': 0.535, 'pos': 0.0}

**Notes:-** As we see that their is no positive score its just neutral and negative so happen is the compound score of zero would be completly negative.

**Conclusion:-** so we can say that the compound score of zero is completely neutral , a compound score above zero can indicate some sort of positive score compund score below zero indicate negative score

#VADER to analyze twitter's reviews

In [12]:
df_data = df.copy()

In [13]:
df_data.head()

,Unnamed: 0,airline_sentiment,name,text,tweet_created,tweet_location,user_timezone,hashtags,tags
0,0,neutral,cairdin,virginamerica dhepburn said,24/02/15 11:35,NaN,Eastern Time (US & Canada),[],"['virginamerica', 'dhepburn']"
1,1,positive,jnardino,virginamerica plus youve added commercials exp...,24/02/15 11:15,NaN,Pacific Time (US & Canada),[],['virginamerica']
2,2,neutral,yvonnalynn,virginamerica didnt today must mean need take ...,24/02/15 11:15,Lets Play,Central Time (US & Canada),[],['virginamerica']
3,3,negative,jnardino,virginamerica really aggressive blast obnoxiou...,24/02/15 11:15,NaN,Pacific Time (US & Canada),[],['virginamerica']
4,4,negative,jnardino,virginamerica really big bad thing,24/02/15 11:14,NaN,Pacific Time (US & Canada),[],['virginamerica']


In [14]:
#here we find the negative,neutral,positive values that are labeled in ouw data set 
df["airline_sentiment"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

Note:-In our model we can see that the negative label are more than the other labels

In [15]:
# we drop the further Columns that are not used in Vader analysise we use it for further analysis
new_df=df_data.drop(columns=["name",'tweet_location', 'user_timezone', 'hashtags','tags',"tweet_created",'Unnamed: 0',])

In [16]:
new_df

,airline_sentiment,text
0,neutral,virginamerica dhepburn said
1,positive,virginamerica plus youve added commercials exp...
2,neutral,virginamerica didnt today must mean need take ...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing
...,...,...
14635,positive,americanair thank got different flight chicago
14636,negative,americanair leaving 20 minutes late flight war...
14637,neutral,americanair please bring american airlines bla...
14638,negative,americanair money change flight dont answer ph...


In [17]:
new_df.dropna(inplace=True)

In [18]:
new_df.head()

,airline_sentiment,text
0,neutral,virginamerica dhepburn said
1,positive,virginamerica plus youve added commercials exp...
2,neutral,virginamerica didnt today must mean need take ...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing


In [19]:
new_df.shape

(14640, 2)

In [20]:
df["airline_sentiment"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [21]:
df_data.iloc[15]["text"]

'virginamerica sfopdx schedule still mia'

In [22]:
sid.polarity_scores(df_data.iloc[15]["text"])

{'compound': -0.296, 'neg': 0.355, 'neu': 0.645, 'pos': 0.0}

In [23]:
new_df["score"] = new_df["text"].apply(lambda text: sid.polarity_scores(text))

In [24]:
new_df.head()

,airline_sentiment,text,score
0,neutral,virginamerica dhepburn said,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,positive,virginamerica plus youve added commercials exp...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,neutral,virginamerica didnt today must mean need take ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,negative,virginamerica really aggressive blast obnoxiou...,"{'neg': 0.345, 'neu': 0.468, 'pos': 0.187, 'co..."
4,negative,virginamerica really big bad thing,"{'neg': 0.486, 'neu': 0.514, 'pos': 0.0, 'comp..."


In [25]:
# now we splite the compound score 
new_df["compound"]=new_df["score"].apply(lambda d:d['compound'])

In [26]:
new_df.head()

,airline_sentiment,text,score,compound
0,neutral,virginamerica dhepburn said,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,positive,virginamerica plus youve added commercials exp...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,neutral,virginamerica didnt today must mean need take ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
3,negative,virginamerica really aggressive blast obnoxiou...,"{'neg': 0.345, 'neu': 0.468, 'pos': 0.187, 'co...",-0.3306
4,negative,virginamerica really big bad thing,"{'neg': 0.486, 'neu': 0.514, 'pos': 0.0, 'comp...",-0.5829


In [27]:
# here we give the lables to the data on the bases of commpund score
def labels(score):
  if score >0:
    return "positive"
  elif score<0:
    return "negative"
  else:
    return"neutral"
new_df['comp_score']=new_df['compound'].apply(labels)


In [28]:
new_df

,airline_sentiment,text,score,compound,comp_score
0,neutral,virginamerica dhepburn said,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
1,positive,virginamerica plus youve added commercials exp...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
2,neutral,virginamerica didnt today must mean need take ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
3,negative,virginamerica really aggressive blast obnoxiou...,"{'neg': 0.345, 'neu': 0.468, 'pos': 0.187, 'co...",-0.3306,negative
4,negative,virginamerica really big bad thing,"{'neg': 0.486, 'neu': 0.514, 'pos': 0.0, 'comp...",-0.5829,negative
...,...,...,...,...,...
14635,positive,americanair thank got different flight chicago,"{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'comp...",0.3612,positive
14636,negative,americanair leaving 20 minutes late flight war...,"{'neg': 0.279, 'neu': 0.721, 'pos': 0.0, 'comp...",-0.7003,negative
14637,neutral,americanair please bring american airlines bla...,"{'neg': 0.0, 'neu': 0.685, 'pos': 0.315, 'comp...",0.3182,positive
14638,negative,americanair money change flight dont answer ph...,"{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'comp...",0.3818,positive


# Here we check the lables that we are find is how exactly match to the pridicted labels

In [29]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [30]:
accuracy_score(new_df['airline_sentiment'],new_df['comp_score'])

0.453551912568306

**Note:-**The accuracy score we get an accuracy of **0.45** 

In [31]:
print(classification_report(new_df['airline_sentiment'],new_df['comp_score']))

              precision    recall  f1-score   support

    negative       0.90      0.38      0.53      9178
     neutral       0.39      0.34      0.36      3099
    positive       0.26      0.89      0.41      2363

    accuracy                           0.45     14640
   macro avg       0.52      0.54      0.43     14640
weighted avg       0.69      0.45      0.48     14640



* **Conclusion:-** It is little bit harder to read and sometimes its also sarcastic which means its hard to detect so sarcasm in something like vader*

#We also  find the Confusion matrix

In [32]:
print(confusion_matrix(new_df['airline_sentiment'],new_df['comp_score']))

[[3482 1471 4225]
 [ 322 1044 1733]
 [  82  167 2114]]
